# auxiliar

In [1]:
import pandas as pd

In [2]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 22.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 35.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 39.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycorrector: filename=pycorrector-0.5.0-py3-none-any.whl size=4480931 sha256=dbc17e88a82dae410386ea038df9f47b2a2783c832f37b20e168a7a3de8e20f0
  Stored in directory: /root/.cache/pip/wheels/03/cd/34/3c82eb07a21023abd3b8c334a96cb2b8f32cdd986d0f7d0bf4
  Created wheel for kenlm: filename=kenlm-0.1-cp310-cp310-linux_x86_64.whl size=3003781 sha256=28e065efd67c7444f771c008ebf570e5ee146e279f3e1e061c33d92e2577af8c
  Stored in directory: /root/.cache/pip/wheels/4e/3a/01/9105a071c30781823efbd96a58279c16f948a87cafb1144042
Successfully built pycorrector kenlm
     ━━━━━━━━━━━

In [6]:
tipificacion=pd.read_csv("/content/PRIORIZAR/data/entrana_retipi_7_junio.csv",delimiter=";").drop_duplicates()


tipificacion = tipificacion[tipificacion['correccion'] != 'conceptos facturados']

tipificacion=tipificacion.reset_index()


tipificacion


,index,NUMERO_PQR,DESCRIPCION_PQR_x,correccion
0,0,MDM-PQR-35550339,Linea:6012167591 Nombre de quien se comunica:...,BENEFICIO DE RETENCION
1,1,MDM-PQR-35550340,CATHERIN GOMEZ SALAZAR 3017869468 catherin...,BENEFICIO DE RETENCION
2,2,MDM-PQR-35550341,CLIENTE SE COMUNICA DEBIDO A QUE EL SERVICIO D...,BENEFICIO DE RETENCION
3,3,MDM-PQR-35550342,DIVA GONZALEZ 38253180 6012695479 317445637...,BENEFICIO DE RETENCION
4,4,MDM-PQR-35550343,GLORIA AYDEE BELTRAN 51742212 luzca_26@hot...,BENEFICIO DE RETENCION
...,...,...,...,...
1055,1057,MDM-PQR-36544676,de la compmesacion de las fallas de junio,COMPENSACION
1056,1058,MDM-PQR-36554753,clietne consutla por ajuste de facturacion,COMPENSACION
1057,1059,MDM-PQR-36721878,Linea: 12054777127 Cedula: 1012396734 Nombre: ...,COMPENSACION
1058,1060,MDM-PQR-36990299,cliente consulta sus ajustes por fallas de ser...,COMPENSACION


In [ ]:
tipificacion['correccion'].value_counts()

In [ ]:
import pandas as pd

import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification





nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing

# aca empieza el modelo

In [ ]:
sys.path.append('../librerias')


In [ ]:
from clean_text import clean_text

In [ ]:

#declare answers and questions
descripcion=tipificacion.DESCRIPCION_PQR_x	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#clean_descripcion=[k for k in clean_descripcion if 'soporte'  in k]

clean_descripcion  [881]

spelling

In [ ]:
from spelling import correct_sentence


corrected_text_list = [correct_sentence(text) for text in clean_descripcion]
clean_descripcion=corrected_text_list

In [ ]:
clean_descripcion[881]

lemma token 

In [ ]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(clean_descripcion)
clean_descripcion[3]

In [ ]:
merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion'])


from unidecode import unidecode

def remove_accents(text):
    return unidecode(text)


# Aplicar la función a la columna 'Texto'
merged_df['descripcion'] = merged_df['descripcion'].apply(remove_accents)


tipificacion=pd.concat([tipificacion, merged_df], axis=1).loc[:, ['descripcion', 'correccion']]

#merged_df = pd.merge(merged_df,leo , on='index', how='inner').loc[:, ['descripcion', 'correccion']]
#############################################




tipificacion = tipificacion[tipificacion['correccion'] != 'SVAS']


tipificacion=tipificacion.dropna()

tipificacion

In [ ]:
tipificacion.isnull().sum()

In [ ]:
tipificacion['correccion'].unique()

In [ ]:
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Assuming you have a DataFrame called 'data' with 'categories' column
# where categories are labeled as 1, 2, and 3
mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compensacin': 4}

# Separate features and target variable
X = tipificacion.drop('correccion', axis=1)
y = tipificacion['correccion'].replace(mapping)

# Oversampling using RandomOverSampler
oversampler = RandomOverSampler(sampling_strategy='auto')
X_resampled, y_resampled = oversampler.fit_resample(X, y)






mapping = {'otros': 0, 'INCREMENTO DE TARIFA': 1, 'ASEGURAMIENTO': 2,  'BENEFICIO DE RETENCION': 3,'compensacin': 4}

crear matriz de numeros 

In [ ]:
from tfidf import TFIDF

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump


X = X_resampled['descripcion']

y = y_resampled




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.95  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_retipificacion.joblib")



In [ ]:

pip install optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score



In [ ]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 5,
        "n_estimators": trial.suggest_int("n_estimators", 20, 90),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 50),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    
    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [ ]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)


In [ ]:
print('Best parameters:', study.best_params)

In [ ]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

In [ ]:
y_pred = mode.predict(X_test_new)


In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from joblib import dump


In [ ]:
dump(value=mode, filename="/content/PRIORIZAR/modelo/retipificacion.joblib")